In [60]:
import os
import numpy as np
import cv2
import easyocr
import assemblyai as aai
from moviepy.video.io.VideoFileClip import VideoFileClip
from collections import Counter

In [61]:
aai.settings.api_key = "065e13309e1e4057b820b51d246d3a32"
reader = easyocr.Reader(['en'])

In [63]:
def transcribe_audio(audio_file_path):
    # Configure transcription with speaker labels
    config = aai.TranscriptionConfig(
        speaker_labels=True
    )

    # Transcribe the audio
    transcript = aai.Transcriber().transcribe(audio_file_path, config)
    # Process and format the results
    results = []
    for utterance in transcript.utterances:
        start = (utterance.start) / (1000)
        end = (utterance.end) / (1000)
        result = (start, end, utterance.speaker, utterance.text)
        results.append(result)
    return results

In [64]:
def select_meeting_type():
    while True:
        print("Select meeting type:")
        print("1. Google Meet")
        print("2. Zoom")
        choice = input("Enter your choice (1 or 2): ")
        if choice in ['1', '2']:
            return choice
        print("Invalid choice. Please try again.")

In [65]:
def process_contour_region(frame, x, y, w, h, prev_text=""):
    # Crop region of interest
    roi = frame[y:y+h, x:x+w]

    # Perform OCR using EasyOCR

    results = reader.readtext(roi)
    # Extract and concatenate detected text
    text = ' '.join([result[1] for result in results])
    
    # Only print if text has changed and is not empty
    if text.strip() and text.strip() != prev_text.strip():
        # print(f"Text Detected: {text.strip()}")
        return text.strip()
    return prev_text

In [67]:
def main():
    meeting_type = select_meeting_type()
    video_file = "samples/green.mp4"
    cap = cv2.VideoCapture(video_file)
    prev_text = ""  # Store previous text
    # audio_file = video_to_audio(video_file)
    transcription_results = transcribe_audio(video_file)

    final_transcript = []

    for start_time, end_time, speaker, text in transcription_results:
        # Calculate the duration
        duration = end_time - start_time
        # Initialize a counter for detected texts
        text_counter = Counter()
        if duration > 20:
            # Analyze only the first 10 seconds
            analysis_duration = 10
            analysis_end_time = start_time + analysis_duration
        else:
            # Analyze the full duration
            analysis_duration = duration
            analysis_end_time = end_time

        # Check every second within the analysis duration
        for t in np.arange(start_time, analysis_end_time, 1.0):
            cap.set(cv2.CAP_PROP_POS_MSEC, t * 1000)  # Convert to milliseconds
            ret, frame = cap.read()

            if not ret:
                continue
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            if meeting_type == '1':  # Google Meet
                lower_blue = np.array([100,50,50])
                upper_blue = np.array([130,255,255])
                mask = cv2.inRange(hsv, lower_blue, upper_blue)

            elif meeting_type == '2':  # Zoom
                lower_green = np.array([40,50,50])
                upper_green = np.array([80,255,255])
                mask = cv2.inRange(hsv, lower_green, upper_green)

            cnts = cv2.findContours(mask.copy(),
                                   cv2.RETR_EXTERNAL,
                                   cv2.CHAIN_APPROX_SIMPLE)[-2]

            if len(cnts) > 0:
                area = max(cnts, key=cv2.contourArea)
                (x,y,w,h) = cv2.boundingRect(area)
                cv2.rectangle(frame,(x,y),(x+w, y+h),(255,0,0),2)
                detected_text = process_contour_region(frame, x, y, w, h, prev_text)
                text_counter[detected_text] += 1  # Count occurrences
                prev_text = detected_text

        # Determine the most common detected text
        if text_counter:
            most_common_text, _ = text_counter.most_common(1)[0]
        else:
            most_common_text = "Unknown Speaker"
        # Append to final transcript
        final_transcript.append(f"Start: {start_time}\n{most_common_text}: {text}\nEnd: {end_time}")

    # Print the final transcript
    for transcript in final_transcript:
        print(transcript)
        print()
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Select meeting type:
1. Google Meet
2. Zoom
Error correcting name 'Cr Andrew Brown': 429 Resource has been exhausted (e.g. check quota).
Error correcting name 'Cr Andrew Brown': 429 Resource has been exhausted (e.g. check quota).
Error correcting name 'Cr Andrew Brown': 429 Resource has been exhausted (e.g. check quota).
Error correcting name 'Cr Andrew Brown': 429 Resource has been exhausted (e.g. check quota).
Error correcting name 'Cr Bruce Thomas': 429 Resource has been exhausted (e.g. check quota).
Error correcting name 'Cr Andrew Browh': 429 Resource has been exhausted (e.g. check quota).
Error correcting name 'Cr Andrew Brown': 429 Resource has been exhausted (e.g. check quota).
Error correcting name 'Cr Andrew Browh': 429 Resource has been exhausted (e.g. check quota).
Error correcting name 'Hvantehty jason dawson': 429 Resource has been exhausted (e.g. check quota).
Error correcting name 'Cr Andrew Brown': 429 Resource has been exhausted (e.g. check quota).
Error correcting na